Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.




In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [5]:
df.sample(2)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
9173,5,OTHER,01 ONE FAMILY DWELLINGS,1,1703,61,NaN,A5,83 WILCOX STREET,NaN,10303.0,1.0,0.0,1.0,"2,530",1472.0,1978.0,1,A5,216500,02/14/2019
7593,3,OTHER,02 TWO FAMILY DWELLINGS,1,7095,8,NaN,B1,90 AVENUE T,NaN,11223.0,2.0,0.0,2.0,"2,000",2183.0,1950.0,1,B1,0,02/07/2019


In [6]:
df.shape

(23040, 21)

### Further filtering of the dataset

Use a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' and the sale price was more than 100 thousand and less than 2 million.

In [7]:
df.dtypes

BOROUGH                            object
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
EASE-MENT                         float64
BUILDING_CLASS_AT_PRESENT          object
ADDRESS                            object
APARTMENT_NUMBER                   object
ZIP_CODE                          float64
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                   object
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                        float64
TAX_CLASS_AT_TIME_OF_SALE           int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
SALE_DATE                          object
dtype: object

In [0]:
df['BUILDING_CLASS_CATEGORY'] = df['BUILDING_CLASS_CATEGORY'].astype(str)

In [0]:
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)


In [0]:
df = df[(df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') & 
        (df['SALE_PRICE'] > 100000) & 
        (df['SALE_PRICE'] < 2000000)] 

- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.

In [0]:
# filtering the dataset by when it was created 
df = df.set_index(df['SALE_DATE'])
df = df.sort_index()

In [15]:
# setting it to January — March 2019 to train. Use data from April 2019 to test.
train_2019 = df['2019-01-01':'2019-03-30']
test_2019 = df['2019-04-01':'2019-04-30']
train_2019.shape, test_2019.shape

((2507, 21), (644, 21))

- [ ] Do one-hot encoding of categorical features.

In [0]:
# TODO
target = 'SALE_PRICE'
y_train = train_2019[target]
y_test = test_2019[target]

In [18]:
df.sample(4)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
SALE_DATE,,,,,,,,,,,,,,,,,,,,,
2019-02-27,4,FLUSHING-NORTH,01 ONE FAMILY DWELLINGS,1,5758,33,NaN,A5,163-60 22ND AVENUE,NaN,11357.0,1.0,0.0,1.0,"2,250",1296.0,1950.0,1,A5,750000,2019-02-27
2019-01-17,4,FOREST HILLS,01 ONE FAMILY DWELLINGS,1,3276,12,NaN,A5,44 SLOCUM CRESCENT,NaN,11375.0,1.0,0.0,1.0,"1,751",1765.0,1925.0,1,A5,1675000,2019-01-17
2019-02-01,2,OTHER,01 ONE FAMILY DWELLINGS,1,4477,14,NaN,A1,2412 FENTON AVENUE,NaN,10469.0,1.0,0.0,1.0,"5,000",1080.0,1915.0,1,A1,430000,2019-02-01
2019-03-20,5,OTHER,01 ONE FAMILY DWELLINGS,1,5603,61,NaN,A2,416 WAINWRIGHT AVENUE,NaN,10312.0,1.0,0.0,1.0,"4,000",1848.0,1975.0,1,A2,560000,2019-03-20


In [20]:
train_2019['NEIGHBORHOOD'].unique()


array(['OTHER', 'FLUSHING-NORTH', 'BEDFORD STUYVESANT', 'FOREST HILLS',
       'BOROUGH PARK', 'ASTORIA'], dtype=object)

In [22]:
train_2019['BUILDING_CLASS_AT_TIME_OF_SALE'].unique()


array(['A9', 'A1', 'A5', 'A0', 'A2', 'A3', 'S1', 'A4', 'A6', 'A8', 'S0'],
      dtype=object)

KeyError: ignored

In [0]:

# TODO
import category_encoders as ce 
features = 'NEIGHBORHOOD'

X_train = train_2019[features]
X_test = test_2019[features]
encoder = ce.one_hot.OneHotEncoder(use_cat_names=True)
X_train_enc = encoder.fit_transform(X_train)
X_test_enc = encoder.transform(X_test) # not fit_transform

In [27]:
X_train_enc

,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA
SALE_DATE,,,,,,
2019-01-01,1,0,0,0,0,0
2019-01-01,1,0,0,0,0,0
2019-01-02,1,0,0,0,0,0
2019-01-02,1,0,0,0,0,0
2019-01-02,1,0,0,0,0,0
...,...,...,...,...,...,...
2019-03-29,1,0,0,0,0,0
2019-03-29,1,0,0,0,0,0
2019-03-29,1,0,0,0,0,0


- [ ] Do feature selection with `SelectKBest`.


In [29]:
features = X_train_enc.columns
n = len(features)
# How many ways to choose 1 to n features?
from math import factorial

def n_choose_k(n, k):
    return factorial(n)/(factorial(k)*factorial(n-k))

combinations = sum(n_choose_k(n,k) for k in range(1,n+1))
print(f'{combinations:,.0f}')

63


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
#TODO